# Create object LBP signatures - frequency counted
This code attempts to combine the LBP numbers for (by doing a frequency count across them) individual object (simulated nuclei). This one is doing this with one property difference

In [53]:
import matplotlib.pyplot as plt
import numpy as np
import math
from PIL import Image
import cv2
import skimage
import tifffile
import os
import pandas as pd
import napari
print(napari.__version__)
from dask_image.imread import imread
import dask.array as da
import io

import platform
if platform.system() == 'Windows':
    fileroot = 'X:/'
    print('Loading Windows')
if platform.system() == 'Linux':
    fileroot = '/home/jovyan/'
    print('Loading Linux')
if os.path.exists('/home/ubuntu/Documents/'):
    fileroot = '/home/ubuntu/Documents/'

colors_new=['#0072b2','#d55e00','#009e73', '#cc79a7','#f0e442','#56b4e9']

import sys
sys.path.append(fileroot+'VU_TEX/Ben_Utilities/')
import useful_functions as uf
import numba_funcs as nf

0.4.12
Loading Linux


In [54]:
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [55]:
maxi = 3328 #this is the maximum value that the LBP could get to

In [56]:
from tqdm import tqdm
from scipy import ndimage as ndi
from functools import partial
from copy import deepcopy
from itertools import product
from datetime import datetime
from tqdm import trange

### Loading the data

In [57]:
directory_originals = [fileroot + 'VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-20b_Initial_image_conversions/']
filenames_originals = ['HnE_norm_V10S15_scale_1.npy']
fullpaths_originals = [x + y for x,y in zip(directory_originals, filenames_originals)]
fullpaths_originals

['/home/jovyan/VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-20b_Initial_image_conversions/HnE_norm_V10S15_scale_1.npy']

In [58]:
directory_lbp_top = [fileroot + 'VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-21_LBP_full_image/']
print(directory_lbp_top)
#should be aligned with fullpaths_originals

['/home/jovyan/VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-21_LBP_full_image/']


In [59]:
directory_gt = fileroot + 'VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-20b_Initial_image_conversions/2021-12-20_Groundtruths/'
filename_gt = 'HnE_norm_V10S15_full_labeled_size_1_resolution_200.npy'
fullpaths_gts = [directory_gt + filename_gt]
print(fullpaths_gts)

['/home/jovyan/VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-20b_Initial_image_conversions/2021-12-20_Groundtruths/HnE_norm_V10S15_full_labeled_size_1_resolution_200.npy']


In [60]:
directory_patches_indices = fileroot + 'VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-20b_Initial_image_conversions/2021-12-20_Patches/'
filename_patches_indices = 'HnE_norm_V10S15_patchsize_200_indices_from_1.npy'
fullpaths_patches_indices = [directory_patches_indices + filename_patches_indices]

In [61]:
directory_out = fileroot + 'VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-21_LBP_pipeline/2021-12-21_Object_info_patchsize_200/'
filename_out_base = 'HnE_norm_V10S15'

In [62]:
#df = pd.read_csv(directory_original_csv + filename_original_csv, index_col=[0])
#df['ImageIndex'] = [uf.getvaluefromstringbest(x, variable='Index', mydtype=int) for x in df['DoubleOutputFilename']]

df_files = pd.DataFrame()
df_files['Fullpaths'] = fullpaths_originals
df_files['Filename'] = filenames_originals
df_files['Shape'] = [uf.get_dims_from_numpy_file(x) for x in df_files['Fullpaths']]
df_files['LBP_Top'] = directory_lbp_top
df_files['Groundtruth'] = fullpaths_gts
df_files['Skip_gt_value'] = [[0]]
df_files['Patches_Indices'] = fullpaths_patches_indices
df_files

,Fullpaths,Filename,Shape,LBP_Top,Groundtruth,Skip_gt_value,Patches_Indices
0,/home/jovyan/VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-20b_Initial_image_conversions/HnE_norm_V10S15_scale_1.npy,HnE_norm_V10S15_scale_1.npy,"(14000, 14500)",/home/jovyan/VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-21_LBP_full_image/,/home/jovyan/VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-20b_Initial_image_conversions/2021-12-20_Groundtruths/HnE_norm_V10S15_full_labeled_size_1_resolution_200.npy,[0],/home/jovyan/VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-20b_Initial_image_conversions/2021-12-20_Patches/HnE_norm_V10S15_patchsize_200_indices_from_1.npy


In [63]:
radius_df = pd.DataFrame(columns=['radii']); 
radius_list = [[each] for each in range(1,2)]
#radius_list = [[each] for each in range(1,11)] + [[5*each] for each in range(3,21)]
for x in radius_list:
    radius_df = radius_df.append({'radii':x}, ignore_index=True)

radius_df['npoints'] = [list(np.asarray((32/5*np.array(x)), dtype=int)) for x in radius_df['radii']]
display(radius_df)

method_df = pd.DataFrame(columns=['METHOD']); 
method_df['METHOD'] = [['uniform']*len(y) for y in radius_df['radii']]
display(method_df)

#df_props = uf.create_df_cross_for_separate_dfs(method_df, radius_df)
df_props = pd.merge(method_df, radius_df, left_index=True, right_index=True)
df_props

,radii,npoints
0,[1],[6]


,METHOD
0,[uniform]


,METHOD,radii,npoints
0,[uniform],[1],[6]


In [64]:
df = uf.create_df_cross_for_separate_dfs(df_files, df_props)
def get_LBP_individual_files(LBP_top, method_list, radii_list, npoints_list):
    fullpaths_lbp = [LBP_top + 'METHOD_' + str(m) + '_npoints_' + str(n)+ '_radius_' + str(r) + '/'
                     'HnE_norm_V10S15_scale_1.npy'
                     for m,r,n in zip(method_list, radii_list, npoints_list)]
    return fullpaths_lbp

df['Fullpaths_LBP'] = list(map(get_LBP_individual_files, 
                               df['LBP_Top'], df['METHOD'], df['radii'], df['npoints']))

#def get_max_LBP_value_for_each_row(list_of_LBP_fullpaths):
#    overall_max = 0
#    for i in trange(len(list_of_LBP_fullpaths)):
#        myarray = np.load(list_of_LBP_fullpaths[i])
#        current_max = np.max(myarray)
#        if current_max > overall_max:
#            overall_max = deepcopy(current_max)
#    return overall_max

def get_LBP_dtype(list_of_LBP_fullpaths):
    output_dtypes = []
    for i in trange(len(list_of_LBP_fullpaths)):
        myarray = np.load(list_of_LBP_fullpaths[i], mmap_mode='r')
        this_dtype = myarray.dtype
        output_dtypes.append(this_dtype)
    return output_dtypes

#df['Max_LBP_value'] = list(map(get_max_LBP_value_for_each_row,df['Fullpaths_LBP']))
df['LBP_dtypes'] = list(map(get_LBP_dtype,df['Fullpaths_LBP']))
df['Fullpath_out_base'] = directory_out + filename_out_base


#this should be commented out if all data points are wanted
#df = deepcopy(df.loc[df['ImageIndex1'] < 25])
df

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 24.44it/s]


,Fullpaths,Filename,Shape,LBP_Top,Groundtruth,Skip_gt_value,Patches_Indices,METHOD,radii,npoints,Fullpaths_LBP,LBP_dtypes,Fullpath_out_base
0,/home/jovyan/VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-20b_Initial_image_conversions/HnE_norm_V10S15_scale_1.npy,HnE_norm_V10S15_scale_1.npy,"(14000, 14500)",/home/jovyan/VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-21_LBP_full_image/,/home/jovyan/VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-20b_Initial_image_conversions/2021-12-20_Groundtruths/HnE_norm_V10S15_full_labeled_size_1_resolution_200.npy,[0],/home/jovyan/VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-20b_Initial_image_conversions/2021-12-20_Patches/HnE_norm_V10S15_patchsize_200_indices_from_1.npy,[uniform],[1],[6],[/home/jovyan/VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-21_LBP_full_image/METHOD_uniform_npoints_6_radius_1/HnE_norm_V10S15_scale_1.npy],[uint16],/home/jovyan/VU_TEX/playground_Ben/2021-12-20_Thymus/2021-12-21_LBP_pipeline/2021-12-21_Object_info_patchsize_200/HnE_norm_V10S15


### Get object properties

In [65]:
patchsize = 200

for q, j in enumerate(range(len(df))):
    output_list = []
    this_fullpath_original = df['Fullpaths'][j]
    fname = df['Filename'][j]
    Shape = df['Shape'][j]
    print(fname, Shape)
    print(q, '/', len(df))
    
    if False: #this allow you to see the original file
        myoriginal = np.load(this_fullpath_original)
        plt.imshow(myoriginal)
        plt.colorbar()
        plt.show()
    
    #This loads the LBP related stuff
    mylbplist = [np.load(x) for x in df['Fullpaths_LBP'][j]]
#    print(mylbplist)
    mylbparray = np.zeros([Shape[0],Shape[1], len(mylbplist)], dtype=np.uint16)
    for p in range(len(mylbplist)):
        mylbparray[:,:,p] = mylbplist[p]
    maxi = np.max(mylbparray) + 1
    
    #This loads the groundtruth
    mygt = np.load(df['Groundtruth'][j])
    
    #This loads the patches arraylist (they should start at 1)
    patches = np.load(df['Patches_Indices'][j])
    
    #This iterates over the patches
    for i_patch in tqdm(patches):
        this_gt = mygt[i_patch[1]:i_patch[2], i_patch[3]:i_patch[4]]
#        plt.imshow(this_gt)
#        plt.colorbar()
#        plt.show()
        counts = np.bincount(this_gt.ravel()) #this bincount is to find groundtruth value
        gt_value = np.argmax(counts)
        if gt_value in df['Skip_gt_value'][j]:
            continue
#        print(gt_value)

        patch_lbp = mylbparray[i_patch[1]:i_patch[2], i_patch[3]:i_patch[4], :]
#        print(patch_lbp.shape)

        lbphist = np.bincount(patch_lbp.ravel(), minlength = maxi)
        
        output_dict = {}
        output_dict.update({'Filename':fname, 'ShapeIDThisImage':i_patch[0], 
                            'LBP_Hist':list(lbphist), 'LBP_size':patch_lbp.size,
                           'OutputCoords':i_patch[1:], 'InputCoords':i_patch[1:],
                           'X0':int(i_patch[1]/patchsize), 'X1':int(i_patch[3]/patchsize),
                           'X0X1':[int(i_patch[1]/patchsize), int(i_patch[3]/patchsize)],
                                  'Groundtruth':gt_value})
        output_list.append(output_dict)
    
    #This is to convert the list into a dataframe and then save it after adding a compressed column
    df_objects = pd.DataFrame(output_list)
#    display(df_objects)
    
    LBParray = np.vstack(df_objects['LBP_Hist'])
#    display(LBParray, LBParray.dtype, LBParray.shape)
    
    non_zero_indices = np.where(LBParray.any(axis=0))[0]
#    display(non_zero_indices)
    
    LBParrayRM = LBParray[:,non_zero_indices] #rm is removed, after removing the things
#    print(LBParrayRM, LBParrayRM.shape)
    
    df_objects['LBP_Hist_Compressed'] = [list(each) for each in LBParrayRM]
    display(df_objects)
    
    timestamp_str = str(datetime.now().strftime("%Y-%m-%d_%H-%M-%S-%f"))
    df_objects.to_csv(df['Fullpath_out_base'][j] + '_objects_radii_' + str(df['radii'][j])+ '_'+ timestamp_str + '.csv')
    df.loc[[j]].to_csv(df['Fullpath_out_base'][j] + '_metadata_radii_' + str(df['radii'][j])+ '_'+ timestamp_str + '.csv')
    np.save(df['Fullpath_out_base'][j] + '_non-zero-indices_radii_' + str(df['radii'][j])+ '_'+
            timestamp_str + '.npy', non_zero_indices)


HnE_norm_V10S15_scale_1.npy (14000, 14500)
0 / 1


100%|█████████████████████████████████████| 5040/5040 [00:01<00:00, 2864.12it/s]


,Filename,ShapeIDThisImage,LBP_Hist,LBP_size,OutputCoords,InputCoords,X0,X1,X0X1,Groundtruth,LBP_Hist_Compressed
0,HnE_norm_V10S15_scale_1.npy,1,"[47, 55, 626, 1060, 1032, 378, 36782, 20]",40000,"[0, 200, 0, 200]","[0, 200, 0, 200]",0,0,"[0, 0]",1,"[47, 55, 626, 1060, 1032, 378, 36782, 20]"
1,HnE_norm_V10S15_scale_1.npy,2,"[48, 49, 669, 1041, 907, 400, 36865, 21]",40000,"[200, 400, 0, 200]","[200, 400, 0, 200]",1,0,"[1, 0]",1,"[48, 49, 669, 1041, 907, 400, 36865, 21]"
2,HnE_norm_V10S15_scale_1.npy,3,"[53, 55, 722, 981, 917, 428, 36820, 24]",40000,"[400, 600, 0, 200]","[400, 600, 0, 200]",2,0,"[2, 0]",1,"[53, 55, 722, 981, 917, 428, 36820, 24]"
3,HnE_norm_V10S15_scale_1.npy,4,"[67, 44, 580, 922, 866, 377, 37116, 28]",40000,"[600, 800, 0, 200]","[600, 800, 0, 200]",3,0,"[3, 0]",1,"[67, 44, 580, 922, 866, 377, 37116, 28]"
4,HnE_norm_V10S15_scale_1.npy,5,"[68, 59, 653, 1005, 923, 446, 36812, 34]",40000,"[800, 1000, 0, 200]","[800, 1000, 0, 200]",4,0,"[4, 0]",1,"[68, 59, 653, 1005, 923, 446, 36812, 34]"
...,...,...,...,...,...,...,...,...,...,...,...
5035,HnE_norm_V10S15_scale_1.npy,5036,"[162, 31, 715, 1207, 2276, 500, 35101, 8]",40000,"[13000, 13200, 14200, 14400]","[13000, 13200, 14200, 14400]",65,71,"[65, 71]",1,"[162, 31, 715, 1207, 2276, 500, 35101, 8]"
5036,HnE_norm_V10S15_scale_1.npy,5037,"[90, 11, 683, 1141, 1222, 390, 36461, 2]",40000,"[13200, 13400, 14200, 14400]","[13200, 13400, 14200, 14400]",66,71,"[66, 71]",1,"[90, 11, 683, 1141, 1222, 390, 36461, 2]"
5037,HnE_norm_V10S15_scale_1.npy,5038,"[162, 82, 614, 1118, 1154, 367, 36471, 32]",40000,"[13400, 13600, 14200, 14400]","[13400, 13600, 14200, 14400]",67,71,"[67, 71]",1,"[162, 82, 614, 1118, 1154, 367, 36471, 32]"
5038,HnE_norm_V10S15_scale_1.npy,5039,"[87, 61, 708, 1030, 1091, 476, 36530, 17]",40000,"[13600, 13800, 14200, 14400]","[13600, 13800, 14200, 14400]",68,71,"[68, 71]",1,"[87, 61, 708, 1030, 1091, 476, 36530, 17]"
